# Notebook 04 — Adaptateurs Bottleneck pour les Langues Africaines

## MMS : Massively Multilingual Speech (Pratap et al., 2023)

**Problème** : Les modèles ASR performants (Whisper, wav2vec 2.0) sont entraînés principalement
sur des langues à haute ressource (anglais, français, espagnol). Les langues africaines
(Swahili, Wolof, Yoruba, Zoulou...) disposent de très peu de données étiquetées.

**Solution MMS** : Utiliser des **adaptateurs bottleneck** — de petits modules insérés dans
un modèle pré-entraîné gelé. Seuls les paramètres des adaptateurs sont entraînés (~2-5% du total).

```
Modèle wav2vec 2.0 pré-entraîné (GELÉ)
    ┌─────────────────────────────────┐
    │  Attention Multi-Têtes (gelée)  │
    │         ↓                       │
    │  ┌─ Adapter ─┐  ← ENTRAÎNABLE  │
    │  │ Down (d→r) │                 │
    │  │ ReLU       │                 │
    │  │ Up   (r→d) │                 │
    │  │ + Résiduel │                 │
    │  └────────────┘                 │
    │         ↓                       │
    │  Feed-Forward (gelé)            │
    │         ↓                       │
    │  ┌─ Adapter ─┐  ← ENTRAÎNABLE  │
    │  │ Down (d→r) │                 │
    │  │ ReLU       │                 │
    │  │ Up   (r→d) │                 │
    │  │ + Résiduel │                 │
    │  └────────────┘                 │
    └─────────────────────────────────┘
```

Ce notebook implémente :
1. L'adaptateur bottleneck (from scratch + PyTorch)
2. L'insertion d'adaptateurs dans un Transformer gelé
3. Une boucle d'entraînement simplifiée avec CTC loss
4. L'évaluation avant/après adaptation avec WER

In [ ]:
import sys
from pathlib import Path

src_path = str(Path("../../src").resolve())
if src_path not in sys.path:
    sys.path.insert(0, src_path)

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt

%matplotlib inline
plt.rcParams['figure.figsize'] = (12, 4)
plt.rcParams['figure.dpi'] = 100

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA disponible: {torch.cuda.is_available()}")

---
## Cell 2 — IMPLÉMENTATION FROM SCRATCH : Adaptateur Bottleneck (NumPy)

L'adaptateur bottleneck est un petit réseau avec 3 étapes :

1. **Down-projection** : `h = x @ W_down` — réduit la dimension de `d_model` à `bottleneck_dim`
2. **Non-linéarité** : `h = relu(h)` — ajoute de la capacité non-linéaire
3. **Up-projection** : `h = h @ W_up` — restaure la dimension originale
4. **Connexion résiduelle** : `output = x + h` — garantit que la shape est préservée

$$\text{Adapter}(x) = x + \text{ReLU}(x W_{\text{down}}) W_{\text{up}}$$

Avec $W_{\text{down}} \in \mathbb{R}^{d \times r}$ et $W_{\text{up}} \in \mathbb{R}^{r \times d}$, où $r \ll d$.

In [ ]:
from audio.adapter import adapter_from_scratch

# Paramètres
d_model = 768       # Dimension du Transformer (wav2vec 2.0 base)
bottleneck_dim = 64  # Dimension réduite de l'adaptateur
batch_size = 2
seq_len = 10

# Créer des données d'entrée aléatoires
np.random.seed(42)
x = np.random.randn(batch_size, seq_len, d_model).astype(np.float32)

# Initialiser les poids de l'adaptateur (Xavier)
scale_down = np.sqrt(2.0 / (d_model + bottleneck_dim))
scale_up = np.sqrt(2.0 / (bottleneck_dim + d_model))
W_down = np.random.randn(d_model, bottleneck_dim).astype(np.float32) * scale_down
W_up = np.random.randn(bottleneck_dim, d_model).astype(np.float32) * scale_up

print(f"Input shape:        {x.shape}")
print(f"W_down shape:       {W_down.shape}  (d_model → bottleneck_dim)")
print(f"W_up shape:         {W_up.shape}  (bottleneck_dim → d_model)")

# Étape par étape
h = np.matmul(x, W_down)
print(f"\nAprès down-projection: {h.shape}  (dimension réduite à {bottleneck_dim})")

h = np.maximum(0, h)  # ReLU
print(f"Après ReLU:            {h.shape}")

h = np.matmul(h, W_up)
print(f"Après up-projection:   {h.shape}  (dimension restaurée à {d_model})")

output = x + h  # Connexion résiduelle
print(f"Après résiduel:        {output.shape}")

# Vérification avec la fonction du module
output_module = adapter_from_scratch(x, W_down, W_up)
assert np.allclose(output, output_module, atol=1e-6)
print(f"\n✓ Shape préservée : input {x.shape} → output {output.shape}")
print(f"✓ Nombre de paramètres adaptateur : {W_down.size + W_up.size:,}")
print(f"  vs paramètres d'une couche Transformer : ~{4 * d_model * d_model:,}")
print(f"  Ratio : {(W_down.size + W_up.size) / (4 * d_model * d_model) * 100:.1f}%")

---
## Cell 3 — IMPLÉMENTATION PYTORCH : BottleneckAdapter (nn.Module)

L'implémentation PyTorch utilise `nn.Linear` pour les projections et `nn.ReLU` pour l'activation.
La connexion résiduelle est identique à la version from scratch.

**Astuce d'initialisation** : Les poids de `up_proj` sont initialisés à zéro pour que
l'adaptateur commence comme une fonction identité (output ≈ input).

In [ ]:
from audio.adapter import BottleneckAdapter

# Créer l'adaptateur PyTorch
adapter = BottleneckAdapter(d_model=768, bottleneck_dim=64)
adapter.eval()

# Afficher l'architecture
print("Architecture BottleneckAdapter :")
print(adapter)

# Compter les paramètres
num_params = sum(p.numel() for p in adapter.parameters())
print(f"\nNombre de paramètres : {num_params:,}")
for name, p in adapter.named_parameters():
    print(f"  {name}: {p.shape} ({p.numel():,} params)")

# Test forward pass
x_torch = torch.randn(2, 10, 768)
with torch.no_grad():
    out_torch = adapter(x_torch)

print(f"\nInput shape:  {x_torch.shape}")
print(f"Output shape: {out_torch.shape}")
print(f"✓ Shape préservée : {x_torch.shape == out_torch.shape}")

# Vérifier que l'initialisation near-zero donne output ≈ input
diff = torch.abs(out_torch - x_torch).mean().item()
print(f"\nDifférence moyenne input/output (init near-zero) : {diff:.6f}")
print("→ L'adaptateur commence comme une quasi-identité !")

---
## Cell 4 — Insertion d'adaptateurs dans un Transformer gelé

La fonction `insert_adapters` :
1. **Gèle** tous les paramètres du modèle de base (`requires_grad = False`)
2. **Insère** un adaptateur après chaque couche d'attention et chaque FFN
3. Seuls les paramètres des adaptateurs restent entraînables

Cela permet d'adapter un modèle pré-entraîné à une nouvelle langue avec très peu de données.

In [ ]:
from audio.adapter import insert_adapters
from architecture.transformer_block import TransformerBlock

# Construire un modèle Transformer simple (simule wav2vec 2.0 Context Network)
class SimpleTransformer(nn.Module):
    def __init__(self, d_model=256, num_heads=4, d_ff=1024, num_layers=4):
        super().__init__()
        self.layers = nn.ModuleList([
            TransformerBlock(d_model, num_heads, d_ff) for _ in range(num_layers)
        ])
    def forward(self, x):
        for layer in self.layers:
            x = layer(x)
        return x

d_model = 256
model = SimpleTransformer(d_model=d_model, num_heads=4, d_ff=1024, num_layers=4)

# Compter les paramètres AVANT insertion
total_before = sum(p.numel() for p in model.parameters())
print(f"Paramètres AVANT insertion : {total_before:,}")
print(f"Tous entraînables : {sum(p.numel() for p in model.parameters() if p.requires_grad):,}")

# Insérer les adaptateurs
bottleneck_dim = 32
model = insert_adapters(model, bottleneck_dim=bottleneck_dim)

# Compter les paramètres APRÈS insertion
total_after = sum(p.numel() for p in model.parameters())
frozen = sum(p.numel() for p in model.parameters() if not p.requires_grad)
trainable = sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f"\nParamètres APRÈS insertion :")
print(f"  Total :       {total_after:,}")
print(f"  Gelés :       {frozen:,} ({frozen/total_after*100:.1f}%)")
print(f"  Entraînables : {trainable:,} ({trainable/total_after*100:.1f}%)")

# Lister les paramètres entraînables
print(f"\nParamètres entraînables (adaptateurs) :")
for name, p in model.named_parameters():
    if p.requires_grad:
        print(f"  {name}: {p.shape}")

# Vérifier que le forward pass fonctionne
x = torch.randn(2, 10, d_model)
model.eval()
with torch.no_grad():
    out = model(x)
print(f"\n✓ Forward pass : {x.shape} → {out.shape}")

---
## Cell 5 — Boucle d'entraînement simplifiée avec CTC Loss

**CTC (Connectionist Temporal Classification)** est la fonction de perte standard pour l'ASR.
Elle permet l'alignement automatique entre la séquence audio et la séquence de caractères,
sans avoir besoin d'un alignement temporel explicite.

Ici, nous simulons un entraînement simplifié :
- Entrée : séquences aléatoires (simulent les sorties du Feature Encoder)
- Cible : séquences de caractères aléatoires
- Seuls les paramètres des adaptateurs sont mis à jour

In [ ]:
# Recréer un modèle frais avec adaptateurs pour l'entraînement
d_model = 256
vocab_size = 30  # Taille du vocabulaire (caractères)

class ASRModelWithAdapters(nn.Module):
    """Modèle ASR simplifié avec Transformer + adaptateurs + projection CTC."""
    def __init__(self, d_model, vocab_size, num_heads=4, d_ff=1024, num_layers=2):
        super().__init__()
        self.transformer = SimpleTransformer(d_model, num_heads, d_ff, num_layers)
        self.ctc_proj = nn.Linear(d_model, vocab_size)  # Projection vers le vocabulaire
    
    def forward(self, x):
        h = self.transformer(x)  # (batch, T, d_model)
        logits = self.ctc_proj(h)  # (batch, T, vocab_size)
        return F.log_softmax(logits, dim=-1)

# Créer le modèle et insérer les adaptateurs
asr_model = ASRModelWithAdapters(d_model, vocab_size)
asr_model = insert_adapters(asr_model, bottleneck_dim=32)

# Rendre la projection CTC entraînable aussi
for p in asr_model.ctc_proj.parameters():
    p.requires_grad = True

trainable_params = [p for p in asr_model.parameters() if p.requires_grad]
print(f"Paramètres entraînables : {sum(p.numel() for p in trainable_params):,}")

# Optimiseur (seulement les paramètres entraînables)
optimizer = torch.optim.Adam(trainable_params, lr=1e-3)
ctc_loss_fn = nn.CTCLoss(blank=0, zero_infinity=True)

# Données synthétiques pour la démonstration
torch.manual_seed(42)
batch_size = 4
T_input = 50   # Longueur séquence d'entrée (pas temporels audio)
T_target = 10  # Longueur séquence cible (caractères)

# Boucle d'entraînement
num_epochs = 20
losses = []

asr_model.train()
for epoch in range(num_epochs):
    # Générer des données synthétiques
    x = torch.randn(batch_size, T_input, d_model)
    targets = torch.randint(1, vocab_size, (batch_size, T_target))  # Éviter le blank (0)
    input_lengths = torch.full((batch_size,), T_input, dtype=torch.long)
    target_lengths = torch.full((batch_size,), T_target, dtype=torch.long)
    
    # Forward pass
    log_probs = asr_model(x)  # (batch, T, vocab_size)
    log_probs = log_probs.transpose(0, 1)  # CTC attend (T, batch, vocab_size)
    
    # Calcul de la perte CTC
    loss = ctc_loss_fn(log_probs, targets, input_lengths, target_lengths)
    
    # Backward pass
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    losses.append(loss.item())
    if (epoch + 1) % 5 == 0:
        print(f"Epoch {epoch+1:3d}/{num_epochs} — CTC Loss: {loss.item():.4f}")

# Visualiser la courbe de perte
plt.figure(figsize=(8, 4))
plt.plot(losses, 'b-', linewidth=2)
plt.xlabel('Epoch')
plt.ylabel('CTC Loss')
plt.title('Entraînement des adaptateurs — Courbe de perte')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print(f"\nPerte initiale : {losses[0]:.4f}")
print(f"Perte finale :   {losses[-1]:.4f}")
print(f"Réduction :      {(1 - losses[-1]/losses[0])*100:.1f}%")

---
## Cell 6 — Évaluation avant/après adaptation avec WER

Pour évaluer l'impact de l'adaptation, nous comparons le WER (Word Error Rate)
avant et après l'entraînement des adaptateurs.

Ici, nous simulons cette comparaison avec des transcriptions synthétiques
pour illustrer le concept. En pratique, on utiliserait un vrai dataset
en langue africaine (ex: FLEURS, CommonVoice).

In [ ]:
from audio.metrics import compute_wer, compute_cer

# Simulation : transcriptions avant/après adaptation
# En pratique, ces transcriptions viendraient du modèle ASR sur un dataset réel

references = [
    "habari yako leo",           # Swahili : "Comment vas-tu aujourd'hui"
    "ninafuraha kukuona",         # Swahili : "Je suis content de te voir"
    "watoto wanacheza uwanjani",  # Swahili : "Les enfants jouent dans le terrain"
    "chakula kiko tayari",        # Swahili : "La nourriture est prête"
]

# Hypothèses AVANT adaptation (erreurs fréquentes)
hyp_before = [
    "habari yako le",             # 'leo' → 'le' (troncation)
    "nina furaha ku kuona",       # Segmentation incorrecte
    "watoto wana cheza uwanja",   # Segmentation + troncation
    "chakula ki ko tayari",       # Segmentation incorrecte
]

# Hypothèses APRÈS adaptation (améliorées)
hyp_after = [
    "habari yako leo",            # Correct !
    "ninafuraha kukuona",         # Correct !
    "watoto wanacheza uwanjani",  # Correct !
    "chakula kiko tayari",        # Correct !
]

print("Évaluation WER et CER — Avant vs Après adaptation")
print("=" * 65)

wer_before_list, wer_after_list = [], []
cer_before_list, cer_after_list = [], []

for i, (ref, hb, ha) in enumerate(zip(references, hyp_before, hyp_after)):
    wb = compute_wer(ref, hb)
    wa = compute_wer(ref, ha)
    cb = compute_cer(ref, hb)
    ca = compute_cer(ref, ha)
    wer_before_list.append(wb)
    wer_after_list.append(wa)
    cer_before_list.append(cb)
    cer_after_list.append(ca)
    print(f"\nPhrase {i+1}: \"{ref}\"")
    print(f"  Avant : \"{hb}\" → WER={wb:.2f}, CER={cb:.2f}")
    print(f"  Après : \"{ha}\" → WER={wa:.2f}, CER={ca:.2f}")

avg_wer_before = np.mean(wer_before_list)
avg_wer_after = np.mean(wer_after_list)
avg_cer_before = np.mean(cer_before_list)
avg_cer_after = np.mean(cer_after_list)

print(f"\n{'=' * 65}")
print(f"WER moyen — Avant : {avg_wer_before:.2f} → Après : {avg_wer_after:.2f}")
print(f"CER moyen — Avant : {avg_cer_before:.2f} → Après : {avg_cer_after:.2f}")

# Visualisation
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

x_pos = np.arange(len(references))
width = 0.35

axes[0].bar(x_pos - width/2, wer_before_list, width, label='Avant', color='#e74c3c', alpha=0.8)
axes[0].bar(x_pos + width/2, wer_after_list, width, label='Après', color='#2ecc71', alpha=0.8)
axes[0].set_xlabel('Phrase')
axes[0].set_ylabel('WER')
axes[0].set_title('Word Error Rate — Avant vs Après adaptation')
axes[0].set_xticks(x_pos)
axes[0].set_xticklabels([f'P{i+1}' for i in range(len(references))])
axes[0].legend()
axes[0].grid(True, alpha=0.3, axis='y')

axes[1].bar(x_pos - width/2, cer_before_list, width, label='Avant', color='#e74c3c', alpha=0.8)
axes[1].bar(x_pos + width/2, cer_after_list, width, label='Après', color='#2ecc71', alpha=0.8)
axes[1].set_xlabel('Phrase')
axes[1].set_ylabel('CER')
axes[1].set_title('Character Error Rate — Avant vs Après adaptation')
axes[1].set_xticks(x_pos)
axes[1].set_xticklabels([f'P{i+1}' for i in range(len(references))])
axes[1].legend()
axes[1].grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

---
## Cell 7 — Discussion : XLS-R et perspectives pour les langues africaines

### XLS-R (Conneau et al., 2020)

XLS-R est l'extension cross-lingue de wav2vec 2.0 :
- **Pré-entraîné** sur 436 000 heures d'audio dans 128 langues
- **Architecture** : wav2vec 2.0 avec des données multilingues
- **Concept clé** : Inventaire phonétique universel — les représentations apprises
  capturent des patterns acoustiques partagés entre langues

### MMS : Massively Multilingual Speech

| Aspect | Détail |
|--------|--------|
| Langues couvertes | 1 100+ |
| Source de données | Textes religieux (Bible, Nouveau Testament) |
| Modèle de base | XLS-R (wav2vec 2.0 multilingue) |
| Adaptation | Adaptateurs bottleneck par langue |
| Paramètres par langue | ~2-5% du modèle total |

### Défis pour les langues africaines

1. **Tonalité** (Yoruba, Igbo) : La hauteur de la voix change le sens lexical
   - Exemple Yoruba : *owó* (argent) vs *owò* (respect) vs *ọwọ́* (main)
   - Métriques adaptées : TER (Tone Error Rate), FER (F0 Error Rate)

2. **Morphologie agglutinante** (Swahili, Zoulou) : Un mot = plusieurs morphèmes
   - Exemple Swahili : *ninakupenda* = ni-na-ku-penda (je-présent-toi-aimer)
   - Impact : Le WER au niveau mot est très sévère

3. **Code-switching** (Swahili-Anglais, Wolof-Français) : Alternance de langues
   - Nécessite des modèles multilingues capables de basculer entre langues

4. **Rareté des données** : Peu de données étiquetées disponibles
   - Solution : Transfer learning avec adaptateurs (ce notebook !)

### Perspectives

- **AfriSpeech** : Datasets dédiés aux langues africaines
- **Fine-tuning multilingue** : Entraîner sur plusieurs langues africaines simultanément
- **Adaptateurs composables** : Combiner des adaptateurs de langues proches
- **Évaluation culturellement adaptée** : Métriques tenant compte de la tonalité et de la morphologie